In [1]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape

from ASCE import load_combos

from steel_design.AISC import aisc
steel_density = 0.2833 #pci

from concrete_design.material_properties import elastic_modulus, cracked_moment_inertia
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf
from concrete_design.beam_calculations import design_properties as bm

from wood_design.ref_design_values.table_4a.table_4a import table_4a_load
table_4a = table_4a_load(species="DF")

<div style="page-break-after: always;"></div>

# Concrete Slab Design
## Inputs

### Materials and Geometry

In [2]:
# Materials
fc = 3000.0 #psi
fy = 60000.0 #psi

# Dimensions
ln = 21 * 12 #in
b = 6 #in
t_slab = 6 #in
cover_b = 1 #in
cover_t = 3/4 #in

In [3]:
# Provided Steel

# bar spacing
n = 1 #bars per width
s = b / n
print(f"bar spacing = {round(s, 2)} in")

# bar properties
bar = "#5"
Ab = rebar.loc[bar, "Area"]
db = rebar.loc[bar, "Diameter"]
d_t = t_slab - cover_b - db/2 #in
d_c = cover_t + db/2 #in
print(f"d_t = {d_t} in")

As = n * Ab #in^2
As_ft = As * 12 / b
print(f"As = {round(As, 3)} in^2 or {round(As_ft, 3)} in^2/ft")

bar spacing = 6.0 in
d_t = 4.6875 in
As = 0.31 in^2 or 0.62 in^2/ft


### Loads

In [4]:
DL = conc_density * t_slab/12
LL = 60 #psf

LN = ln / 12 #ft
D=DL*b/12 #plf
L=LL*b/12 #plf
w_u = load_combos.LRFD(D, L) #plf
print(f"w_u = {round(w_u, 2)} plf")

Mu = w_u * LN**2 / 8 * 12 #lb-in
f"Mu = {round(Mu / 12, 2)} lb-ft"

w_u = 93.0 plf


'Mu = 5126.62 lb-ft'

## Strength Design

In [5]:
# Flexural Strength

a = As * fy / (0.85 * fc * b)
c = a/0.85 #in
ε_t = bm.tension_steel_strain(d_t, c)
φ = bm.phi_factor(ε_t, fy)
print(f"φ = {round(φ, 3)}")
Mn = As * fy * (d_t - a/2)
φMn = φ * Mn

print(f"Mu = {round(Mu / 12, 2)} lb-ft")
print(f"φMn = {round(φMn / 12, 2)} lb-ft")
φMn > Mu

φ = 0.9
Mu = 5126.62 lb-ft
φMn = 5691.12 lb-ft


True

<div style="page-break-after: always;"></div>

## Deflection

### Sustained Load Deflection

#### Short-term Dead Load Deflection

In [6]:
w_D = D #plf

M_D = w_D * LN**2 / 8 * 12 #lb-in
print(f"M_D = {round(M_D / 12, 2)} lb-ft")

M_D = 2067.19 lb-ft


In [7]:
# Stiffness Properties
fr = 7.5 * sqrt(fc) #psi
Ig = b * t_slab**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in")
yt = t_slab / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

Ig = 108.0 in


'Mcr = 1232.38 lb-ft'

In [8]:
def elastic_deflection(Ma, w_a):
    # Elastic Modulus
    Ec = elastic_modulus(fc=fc) #psi
    print(f"Ec = {round(Ec/1000, 2)} ksi")
    # Moment of Inertia
    Ie = Ig
    Icr = cracked_moment_inertia(b=b, d=d_t, Ec=Ec, As=As)
    print(f"Icr = {round(Icr, 2)} in^4")
    if Ma > 2/3*Mcr:
        Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
        print(f"Ie = {round(Ie, 2)} in")
    else:
        print("section uncracked")

    #Elastic Deflection
    defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * Ie)
    return defl_elastic

DL_defl = elastic_deflection(Ma=M_D, w_a = w_D)
f"Elastic Deflection = {round(DL_defl, 3)} in"

Ec = 3320.56 ksi
Icr = 36.14 in^4
Ie = 40.38 in


'Elastic Deflection = 1.224 in'

#### Long-term Dead Load Deflection

In [9]:
# Deflection Factor
xi = 2.0 #duration factor
bar = "#5"
Ab_c = rebar.loc[bar, "Area"] #in^2
As_c = n * Ab_c #compression reinforcement
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 0.583'

In [10]:
# Long-Term Deflection
sustained_defl = DL_defl * lmbda_dlt #in
f"Long-Term Deflection = {round(sustained_defl, 3)} in"

'Long-Term Deflection = 0.713 in'

In [11]:
# Total Sustained Dead Load Deflection
sustained_total = DL_defl + sustained_defl
f"Sustained Dead Load Deflection = {round(sustained_total, 3)} in"

'Sustained Dead Load Deflection = 1.937 in'

### Live Load Deflection

In [12]:
w_L = L #plf

M_L = w_L * LN**2 / 8 * 12 #lb-in
print(f"Ma = {round(M_L / 12, 2)} lb-ft")

Ma = 1653.75 lb-ft


In [13]:
LL_defl = elastic_deflection(Ma=M_L, w_a=w_L)
f"Elastic Deflection = {round(LL_defl, 3)} in"

Ec = 3320.56 ksi
Icr = 36.14 in^4
Ie = 43.24 in


'Elastic Deflection = 0.914 in'

In [14]:
long_term_defl_with_LL = sustained_total + LL_defl
f"sustained deflection + instantaneous LL deflection = {round(long_term_defl_with_LL, 4)} in"

'sustained deflection + instantaneous LL deflection = 2.8513 in'

#### IBC Check

In [15]:
dead_live_limit = ln/240
live_load_limit = ln/360

dead_live_limit, live_load_limit

(1.05, 0.7)

## Minimum Requirements

In [16]:
# Minimum Thickness if deflection is not calculated
t_min = ln / 30
print(f"t_min (two way) = {round(t_min, 2)} in")

t_min = ln / 20
print(f"t_min (one way) = {round(t_min, 2)} in")

t_min (two way) = 8.4 in
t_min (one way) = 12.6 in


In [17]:
# Minimum Steel
Ag = b * t_slab #in^2
As_min = 0.0018 * Ag
As_minft = As_min * 12 / b

f"As,min = {round(As_min, 3)} in^2 or {round(As_minft, 3)} in^2/ft"

'As,min = 0.065 in^2 or 0.13 in^2/ft'